<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="center" width="30%">

## Xarray 4: Remote access

## Overview
1. Work with a multiple-file Xarray `Dataset` hosted on NCAR's Research Data Archive
2. Subset the Dataset along its dimensions
3. Perform unit conversion
4. Create a well-labeled multi-parameter contour plot of gridded ERA-5 reanalysis data

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| Xarray Lessons 1-3| Necessary | |

* **Time to learn**: 30 minutes

## Imports

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime as dt
from metpy.units import units
from metpy import calc as mpcalc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import requests
from pathlib import Path

### Work with the ERA-5 Dataset hosted on NCAR's [Remote Data Archive](https://rda.ucar.edu/datasets/ds633.0/#description)</span>

Datasets are ever-growing in terms of size and sheer number. It is not "remotely" possible to download every dataset one might want to use to your computer! Fortunately, although Xarray provides easy access to files stored "locally" on disk, it can also give you access to files stored on remotely-located servers via different types of *data access protocols*. One such protocol is called **OpenDAP**. It makes files available via a specially-crafted web link, or *URL*. This is how we will access ERA-5 datasets stored on the ERA-5 [RDA's THREDDS server](https://rda.ucar.edu/thredds/catalog/files/g/ds633.0/catalog.html) data catalog.

<div class="alert alert-info">Besides <b>OpenDAP</b>, datasets are increasingly served via cloud providers, such as Amazon Web Services (AWS), Google Cloud Platform (GCP), and Microsoft Azure. We will explore how Xarray can access these datasets in a future lesson.
</div>

Construct OpenDAP URL's for the ERA-5 datasets served by the RDA server.

<div class="alert alert-warning"><b>Note: </b>RDA follows a particular convention in the directory and file names for the ERA-5 dataset. Single-level fields are stored in one monthly file, with a naming convention that includes the string <i>sfc</i>. Fields on multiple levels, such as pressure (isobaric) levels, are stored in daily files. For isobaric levels, the naming convention includes the string <i>pl</i>. In this case, we will retrieve sea-level pressure (SLP) and geopotential on isobaric surfaces, so we will be both of these two strings in our URLs.
</div>

In [ ]:
ds_urlp = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.sfc/201210/e5.oper.an.sfc.128_151_msl.ll025sc.2012100100_2012103123.nc'
ds_urlz = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.0/e5.oper.an.pl/201210/e5.oper.an.pl.128_129_z.ll025sc.2012103000_2012103023.nc'

<div class="alert alert-warning"><b>Note: </b>Since the ERA-5 dataset requires password authorization, we need to perform two steps before we can use Xarray's <code>open_dataset</code> method to access each file.
</div>

#### Step 1: Use your RDA credentials to access these individual NetCDF files. You should have previously registered with RDA, and then saved your user id and password in your home directory as .rdarc , with permissions such that only you have read/write access.

In [ ]:
HOME = str(Path.home())
credFile = open(HOME+'/.rdarc','r')
userId, pw = credFile.read().split()

#### Step 2: Connect to the RDA THREDDS server and point to the files of interest. To deal with the password authentication issue, Xarray provides the `backends.PydapDataStore.open` option.

In [ ]:
session = requests.Session()
session.auth = (userId, pw)
storep = xr.backends.PydapDataStore.open(ds_urlp, session=session)
storez = xr.backends.PydapDataStore.open(ds_urlz, session=session)

<div class="alert alert-warning"><b>Note: </b>If you get an error after the cell, check to be sure you have registered @ RDA and you have set up a <code>.rdarc</code> file in your home directory, with the proper file access permissions.
    
If you are sure that you have done so, it's possible that the RDA THREDDS server might be down.
</div>

Now open the two files as separate Datasets.  (avoid using open_mfdataset as subsetting will cause failures down the line)

<div class="alert alert-warning"><b>Note: </b>As we will see in future lessons, Xarray provides a method, <code>open_mfdataset</code>, that allows for one <code>Dataset</code> to be created from multiple sources. However, this may lead to strange errors after we subset the data later in this notebook. This problem seems specific to using <code>open_mfdataset</code> on file served by THREDDS.
</div>

In [ ]:
dsSLP = xr.open_dataset(storep)
dsZ = xr.open_dataset(storez)

 
Let's see what one of these Datasets looks like.

In [ ]:
dsZ

In [ ]:
dsSLP

In [ ]:
dsSize = dsZ.nbytes / 1e9
print ("Size of dataset = %.3f GB" % dsSize)

<div class="alert alert-info">That's a fairly large dataset! However, Xarray uses what's called <i>lazy loading</i>, in that the actual data transfer only occurs when we actually need the data to perform computations or visualizations. By subsetting the data so we only get what we want, the actual data transfer will be much smaller.
</div>

#### Subset the Dataset along its dimensions

We noticed in the previous notebook that our contour labels were not appearing with every contour line. This is because we passed the entire horizontal extent (all latitudes and longitudes) to the `ax.contour` method. Since our intent is to plot only over a regional subset, we will use the `sel` method on the latitude and longitude dimensions as well as time and isobaric surface.

We'll also use Datetime and string methods to more dynamically assign various dimensional specifications, as well as aid in figure-labeling later on.

In [ ]:
# Areal extent
lonW = -100
lonE = -60
latS = 20
latN = 50
latRange = np.arange(latS-5,latN+5,.25) # expand the data range a bit beyond the plot range
lonRange = np.arange((lonW-5+360),(lonE+5+360),.25) # Need to match longitude values to those of the coordinate variable

# Vertical level specificaton
vlevel = 500
levelStr = str(vlevel)

# Date/Time specification
Year = 2012
Month = 10
Day = 30
Hour = 0
Minute = 0
dateTime = dt(Year,Month,Day, Hour, Minute)
timeStr = dateTime.strftime("%Y-%m-%d %H%M UTC")

# Data variable selection

SLP = dsSLP['MSL'].sel(time=dateTime,latitude=latRange,longitude=lonRange)
Z = dsZ['Z'].sel(time=dateTime,latitude=latRange,longitude=lonRange,level=vlevel)

### Let's look at some of the attributes

In [ ]:
Z.shape

In [ ]:
Z.dims

In [ ]:
Z.units

In [ ]:
SLP.shape

In [ ]:
SLP.dims

In [ ]:
SLP.units

#### Define our subsetted coordinate arrays of lat and lon. Pull them from any of the DataArrays. We'll need to pass these into the contouring functions later on.

In [ ]:
lats = SLP.latitude
lons = SLP.longitude

In [ ]:
lats

In [ ]:
lons

In [ ]:
SLP

#### Perform unit conversions

Traditionally, we plot SLP in units of hectopascals (hPa) and geopotential heights in units of decameters. The SLP units conversion is a bit more straightforward than for Geopotential to Height. We take the DataArray and apply [MetPy's unit conversion method](https://unidata.github.io/MetPy/latest/tutorials/xarray_tutorial.html#units).

In [ ]:
SLP = SLP.metpy.convert_units('hPa')

Next, convert geopotential to height, as we did in the previous notebook. In the same line of code, convert the resulting units from meters to decameters.

In [ ]:
HGHT = mpcalc.geopotential_to_height(Z).metpy.convert_units('dam')

#### Create a well-labeled multi-parameter contour plot of gridded ERA-5 reanalysis data

We will make contour lines of SLP in hPa, contour interval = 4, and filled contours of geopotential height in decameters, contour interval = 6.

As we've done before, let's first define some variables relevant to Cartopy. Recall that we already defined the areal extent up above when we did the data subsetting.

In [ ]:
cLon = -80
cLat = 35

proj_map = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
proj_data = ccrs.PlateCarree() # Our data is lat-lon; thus its native projection is Plate Carree.
res = '50m'

Now define the range of our contour values and a contour interval. 4 hPa is standard for SLP.

In [ ]:
minVal = 900
maxVal = 1080
cint = 4
SLPcintervals = np.arange(minVal, maxVal, cint)
SLPcintervals

In [ ]:
minVal = 468
maxVal = 606
cint = 6
HGHTcintervals = np.arange(minVal, maxVal, cint)
HGHTcintervals

Create a meaningful title string.

In [ ]:
tl1 = "ERA-5 " + levelStr + " hPa geopotential heights (color fill, dam) and SLP (lines, hPa)"
tl2 = str('Valid at: '+ timeStr)
title_line = (tl1 + '\n' + tl2 + '\n')

Plot the map.

In [ ]:
fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1,projection=proj_map)
ax.set_extent ([lonW,lonE,latS,latN])
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))
CF = ax.contourf(lons,lats,HGHT, levels=HGHTcintervals,transform=proj_data,cmap=plt.get_cmap('coolwarm'))
cbar = plt.colorbar(CF,shrink=0.5)
cbar.ax.tick_params(labelsize=16)
cbar.ax.set_ylabel("Height (dam)",fontsize=16)

CL = ax.contour(lons,lats,SLP,SLPcintervals,transform=proj_data,linewidths=1.25,colors='green')
ax.clabel(CL, inline_spacing=0.2, fontsize=11, fmt='%.0f')
title = plt.title(title_line,fontsize=16)

We're missing the outer longitudes at higher latitudes. This is a consequence of reprojecting our data from PlateCarree to Lambert Conformal. We could do one of two things to resolve this:

1. Re-subset our original datset by extending the longitudinal ranges
2. Slightly constrain the map plotting region

Let's use the latter method. The determination of how many degrees longitude to constrain is a process of trial and error; you want to avoid any blank areas in your map, but not restrict your map domain so much that you lose features at the edge of your map.

In [ ]:
constrainLon = 3.3 # trial and error!

fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1,projection=proj_map)
ax.set_extent ([lonW+constrainLon,lonE-constrainLon,latS,latN])
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

CF = ax.contourf(lons,lats,HGHT, levels=HGHTcintervals,transform=proj_data,cmap=plt.get_cmap('coolwarm'))
cbar = plt.colorbar(CF,shrink=0.5)
cbar.ax.tick_params(labelsize=16)
cbar.ax.set_ylabel("Height (dam)",fontsize=16)

CL = ax.contour(lons,lats,SLP,SLPcintervals,transform=proj_data,linewidths=1.25,colors='green')
ax.clabel(CL, inline_spacing=0.2, fontsize=11, fmt='%.0f')
title = plt.title(title_line,fontsize=16)

---
## Summary
* Via the OpenDAP protocol, Xarray can open remotely-served datasets, such as the ERA-5 from the RDA THREDDS server.
* Subsetting our data request results in less data being transferred, as well as more consistent contour labels.

### What's Next?
We'll continue to work with the ERA-5 repository, constructing time series and performing more complicated diagnostic calculations.

## Resources and References
1. [About ERA-5](https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5)
1. [RDA's ERA-5 Documentation](https://rda.ucar.edu/datasets/ds633.0/#!docs)
1. [Data Table for ERA-5 Surface Variables](https://rda.ucar.edu/datasets/ds633.0/docs/ds633.0.e5.oper.an.sfc.grib1.table.web.txt)
1. [Data Table for ERA-5 Pressure Levels Variables](https://rda.ucar.edu/datasets/ds633.0/docs/ds633.0.e5.oper.an.pl.grib1.table.web.txt)
1. [The THREDDS Data Server](https://www.unidata.ucar.edu/software/tds/current/)